In [1]:
import os

os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/nsk20/End-to-End-Data-Science-Pipeline.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "nsk20"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "195455390c4edd1543bee9fc88f5845a90efa779"

In [2]:
%pwd

'/home/user/datascienceproject/research'

In [3]:
os.chdir("../")
%pwd

'/home/user/datascienceproject'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [5]:
from src.datascienceproject.constants import *
from src.datascienceproject.utils.common import read_yaml, create_directories, save_json

Bad pipe message: %s [b' 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Sa']
Bad pipe message: %s [b'ri/537.36\r\nAccept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/', b'ng,*/*;q=0.8\r\nAccept-Encoding: gzip, deflate, br, zstd\r\nAccept-Language: en-US,en;q=0.7\r\nConnect']
Bad pipe message: %s [b'n: keep-alive\r\nReferer: https://studio.firebase.google.com/\r\nSec-Ch-Ua: "Brave";v="143", "Chromium";v="1', b'", "Not A(Brand";v="24"\r\nSec-Ch-Ua-Arch: "x86"\r\nSec', b'h-Ua-Bitness: "64"\r\nSec-Ch-Ua-Full-Version-L', b't: "Brave";v="143.0.0.0", "Chromium";v="143.0.0.0", "Not A(Brand";v="24.0.0.0"\r\nSec-Ch-Ua-Mobile: ?0\r\nSe', b'Ch-Ua-Model: ""\r\nSec-Ch-Ua-Platform: "Windows"\r\nSec-Ch-Ua-Platform-Version: "19.0.0"\r\nSec-Ch-Ua-Wo', b'4: ?0\r\nSec-Fetch-Dest: iframe\r\nSec-Fetch-Mode: navigate\r\nSec-Fetch-Site: cross-site\r\nSec-Fetch-Storage-Access: none\r\nS']
Bad pipe message: %s [b'-Fetch-User: ?1\r\nSec-Gpc: 1\r\nS

In [10]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, 
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN.name

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=str(schema),
            mlflow_uri="https://dagshub.com/nsk20/End-to-End-Data-Science-Pipeline.mlflow"
        )
        return model_evaluation_config


In [7]:
import pandas as pd
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [16]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rsme = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rsme, mae, r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            (rsme, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            #saving metrics as local
            scores = {"rsme": rsme, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rsme", rsme)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                #Register the model
                # There are other ways to use the model registry, which depends on the use case

                mlflow.sklearn.log_model(model, "model", registered_model_name = "ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [17]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2026-01-15 19:29:44,307: INFO: common: yaml file: config/config.yaml loaded successfully]
[2026-01-15 19:29:44,312: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-15 19:29:44,334: INFO: common: yaml file: schema.yaml loaded successfully]
[2026-01-15 19:29:44,344: INFO: common: created directory at: artifacts]
[2026-01-15 19:29:44,351: INFO: common: created directory at: artifacts/model_evaluation]
[2026-01-15 19:29:44,583: INFO: common: json file saved at: artifacts/model_evaluation/metric.json]


2026/01/15 19:29:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/15 19:29:56 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Successfully registered model 'ElasticnetModel'.
2026/01/15 19:29:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.


🏃 View run bustling-hound-130 at: https://dagshub.com/nsk20/End-to-End-Data-Science-Pipeline.mlflow/#/experiments/0/runs/bd3e5cdb75c448a487e4636e7160fa8c
🧪 View experiment at: https://dagshub.com/nsk20/End-to-End-Data-Science-Pipeline.mlflow/#/experiments/0
